In [19]:
import os
import requests
import json
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta


class KnackAFT:
    def __init__(self):
        # API #
        self.API_VERSION = 'v1'
        self.API_KEY = '1a210580-315e-11ea-a6a4-bb031a9e1ba1'
        self.APP_ID = '5e13989941e72c0e039e117f'
        self.CUSTOM_KNACK_ENDPOINT = 'knack.aft.org'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.{self.CUSTOM_KNACK_ENDPOINT}/{self.API_VERSION}/'
        self.LOADER_URL = f'https://loader.{self.CUSTOM_KNACK_ENDPOINT}/{self.API_VERSION}/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
    
    def loader(self):
        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']

            if 'Entity-' in name:
                for item in obj['fields']:
                    fields.update({item['name']:item['key']})
                self.APP_DICT.update({name.replace('Entity-', ''):{'id':key,'fields':fields}})
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
            
    def convert_fields_ids2name(self, object_name, ids):
        field_dict = self.APP_DICT[object_name]['fields']
        out_dict = {}
        
        for k, v in ids.items():
            if 'field' in k:
                if 'raw' in k:
                    newk = k.replace('_raw', '')
                    key = self.get_key(field_dict, newk)
                    if key:
                        out_dict.update({(key+'_raw'):v})
                    else:
                        out_dict.update({k:v})
                else:
                    key = self.get_key(field_dict, k)
                    if key:
                        out_dict.update({key:v})
                    else:
                        out_dict.update({k:v})
            else:
                out_dict.update({k:v})
        return out_dict
    
    def convert_fields_name2ids(self, object_name, names):
        for k, v in names.items():
            if 'field' in k:
                key = self.APP_DICT[object_name]['fields']
            print(k)
            print(v)


In [22]:
# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))


# GLOBAL VARIABLES #
YESTERDAY_DATE = (date.today() - timedelta(days = 1)).strftime("%m/%d/%Y")
ENTITY_LIST = ['LocalDues','Employer','LocalAgreement','Unit','LocalJobClass','JobTitle','WorkLocation','WorkStructure']

client = KnackAFT()
client.loader()
output = client.find_matches('LocalDues', 'mdate', '08/18/2022')

jprint(client.APP_DICT)
# if output:
#     for item in output:
#         t = client.convert_fields_ids2name('LocalDues', item)
#         jprint(t)
#         jprint(client.convert_fields_name2ids('LocalDues', t))





# LIST OF METHODS FROM CLIENT

# - CT.init()
# - CT.loader()
# - CT.getJson(url)
# - CT.getObjectJSON(name)
# - CT.find_matches(object_name, field, value)
# - CT.convert_fields_ids2name(self, object_name, ids)
# - CT.convert_fields_name2ids(self, object_name, names)




# output = C.find_matches('Unit', 'mdate', '08/11/2022')
# output = C.find_matches('Unit', 'mdate', '08/11/2022')
# output = C.find_matches('Unit', 'mdate', '08/11/2022')
# output = C.find_matches('Unit', 'mdate', '08/11/2022')


        
#pl = CLIENT.getObjectPayload('Unit')
#print(json.dumps(output, indent=4)) 
#print(json.dumps(dictout['Entity-Employer'], indent=4))

{
    "StateFed": {
        "id": "object_43",
        "fields": {
            "AffiliateID": "field_525",
            "AffiliateName": "field_526",
            "AffiliateNumber": "field_527",
            "cdate": "field_528",
            "mdate": "field_529",
            "Status": "field_530",
            "TeamMember": "field_531",
            "AffiliateGuid": "field_532",
            "AffiliateAbbreviatedName": "field_533",
            "AffiliateAcronym": "field_534",
            "AffiliateEIN": "field_535",
            "AffiliatePerCapitaPIN": "field_536",
            "BillHighwayGroupId": "field_537",
            "IsChartered": "field_538",
            "CharterDate": "field_539",
            "ParentAffiliateId": "field_540",
            "AffiliateTypeId": "field_541",
            "AffiliateDesignationId": "field_542",
            "AffiliateGeoReachId": "field_543",
            "IsAffiliateActive": "field_544",
            "AffiliateInactiveReasonId": "field_545",
            "Affil

In [ ]:
sorting_dict = {
    'LocalDuesCategory':{
        'extras_array':[False, True, False],
        'sort_array':['User','Action','KnackFieldGUID','LocalDuesCategoryID','LocalDuesCategoryName','StatePerCapitaID','NationalPerCapitaID','LocalDuesAmount','LocalDuesPercentage','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualAffiliateToID','UpdateIndividualAffiliateToName','UpdateIndividualAffiliateToKnackGUID','AffiliateID','ProcessedInAFTDB']
    },
    'Employer':{
        'extras_array':[False, False, True],
        'sort_array':['User','Action','KnackFieldGUID','EmployerID','EmployerName','EmployerTypeID','ParentEmployerID','ParentEmployerName','ParentEmployerKnackGUID','Acronym','EmployerCode','Area','ChapterID','HasPrivateSector','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','AffiliateID','ProcessedInAFTDB']
    },
    'LocalAgreement':{
        'extras_array':[True, False, False],
        'sort_array':['User','Action','KnackFieldGUID','LocalAgreementID','LocalAgreementName','EmployerID','EmployerName','EmployerKnackGUID','LocalAgreementTypeID','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','AffiliateID','ProcessedInAFTDB']
    },
    'Unit':{
        'extras_array':[True, False, False],
        'sort_array':['User','Action','KnackFieldGUID','UnitID','UnitName','UnitTypeID','DivisionID','LocalAgreementID','LocalAgreementName','LocalAgreementKnackGUID','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','AffiliateID','ProcessedInAFTDB']
    },
    'LocalJobClass':{
        'extras_array':[True, True, False],
        'sort_array':['User','Action','KnackFieldGUID','LocalJobClassID','LocalJobClassName','NationaJobClassID','UnitID','UnitName','UnitKnackGUID','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualEmployerToID','UpdateIndividualEmployerToName','UpdateIndividualEmployerToKnackGUID','AffiliateID','ProcessedInAFTDB']
    },
    'JobTitle':{
        'extras_array':[True, True, False],
        'sort_array':['User','Action','KnackFieldGUID','JobTitleID','JobTitleName','LocalJobClassID','LocalJobClassName','LocalJobClassKnackGUID','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualEmployerToID','UpdateIndividualEmployerToName','UpdateIndividualEmployerToKnackGUID','AffiliateID','ProcessedInAFTDB']
    },
    'WorkLocation':{
        'extras_array':[True, True, True],
        'sort_array':['User','Action','KnackFieldGUID','WorkLocationID','WorkLocationName','WorkLocationTypeID','NationalInstitutionTypeID','ParentWorkLocationID','ParentWorkLocationName','ParentWorkLocationKnackGUID','EmployerID','EmployerName','EmployerKnackGUID','WorkLocationCode','WorkLocationArea','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualEmployerToID','UpdateIndividualEmployerToName','UpdateIndividualEmployerToKnackGUID','AffiliateID','ProcessedInAFTDB']
    },
    'WorkStructure':{
        'extras_array':[True, True, True],
        'sort_array':['User','Action','KnackFieldGUID','WorkStructureID','WorkStructureName','WorkStructureTypeID','ParentWorkStructureID','ParentWorkStructureName','ParentWorkStructureKnackGUID','EmployerID','EmployerName','EmployerKnackGUID','WorkStructureCode','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualEmployerToID','UpdateIndividualEmployerToName','UpdateIndividualEmployerToKnackGUID','AffiliateID','ProcessedInAFTDB']
    }
}





def find_records_updated_at_date(knack_object, date):
    #Convert to IDs
    knack_object_id = find_object_id(knack_object)
    field_to_match_id = find_field_id(knack_object, "mdate")
    
    #Get Id
    match_filter = {'match':'and', 'rules':[{'field':field_to_match_id, 'operator':'is', 'value': date}]}
    filter_for_url = urllib.parse.quote(json.dumps(match_filter))
    request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
    
    r = requests.get(url = request_url, headers = GET_HEADERS)
    #print(json.dumps(r.json(), indent=4))
    res_json_dict = json.loads(json.dumps(r.json()))
    if res_json_dict["total_records"] == 0:
        return ''
    else:
        return res_json_dict["records"]

    
def find_connection_payload(name, lookup):
    output = {}
    knack_object_id = find_object_id(name)
    if lookup:
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" + lookup
        r = requests.get(url = request_url, headers = GET_HEADERS)
        output = json.loads(json.dumps(r.json()))
        return output
    else:
        return ''
    

def getFields(record, object_name):
    obj = knackmappingdict[object_name]
    output = {}
    
    conn_values = obj['fields']
    conn_values.update({(object_name+'ID'):obj['ID'], 'KnackFieldGUID':obj['GUID']})
    
    for k,v in conn_values.items():
        if record[v]:
            data = record[v]
        else:
            data = ''
        output.update({k:data})
        
    output['UpdatedAt'] = output.pop('mdate')
    
    if output[(object_name+'ID')]:
        if output['DeletedAt']:
            output.update({'Action':'Delete'})
        else:
            output.update({'Action':'Update'})
    else:
        if output['DeletedAt']:
            output = {}
        else:
            output.update({'Action':'Insert'})
    if output:
        output.update({'ProcessedInAFTDB':'0'})
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def getConnections(record, object_name):
    obj = knackmappingdict[object_name]
    output = {}
    
    conn_values = obj['connections']
    
    for k,v in conn_values.items():
        fieldval = v.replace('_raw', '')
        res = ''
        knack_object = k.replace('Entity-', '')
        if record[fieldval]:
            data = record[v][0]['id']
            field_id = knackmappingdict[knack_object]['ID']
            res = find_connection_payload(knack_object, data)[field_id]
        output.update({(knack_object+'ID'):res})
        
    output['User'] = output.pop('UserID')
    return output
    

def getMappedFields(record, object_name):
    conn_values = knackmappingdict[object_name]['mapped_fields']
    for k,v in conn_values.items():
        entity = k.replace('Entity-', '')
        output = {entity+'ID':'',entity+'Name':'', entity+'KnackGUID':''}
        fieldval = v.replace('_raw', '')
        if record[v]:
            data = record[v][0]['id']
                
            field_id = knackmappingdict[entity]['ID']
            field_guid = knackmappingdict[entity]['GUID']
            field_name = knackmappingdict[entity]['fields'][(entity+'Name')]
                
            res = find_connection_payload(entity, data)
            out_id = res[field_id]
            out_guid = res[field_guid]
            out_name = res[field_name]
                
            output.update({entity+'ID':out_id,entity+'Name':out_name, entity+'KnackGUID':out_guid})
    
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def getParent(record, object_name):
    output = {'Parent'+object_name+'ID':'','Parent'+object_name+'Name':'', 'Parent'+object_name+'KnackGUID':''}
    conn_values = knackmappingdict[object_name]['parent_fields']
    for k,v in conn_values.items():
        if 'Parent' in k:
            fieldval = v.replace('_raw', '')
            if record[fieldval]:
                data = record[v][0]['id']
                
                field_id = knackmappingdict[object_name]['ID']
                field_guid = knackmappingdict[object_name]['GUID']
                field_name = knackmappingdict[object_name]['fields'][(object_name+'Name')]
                
                res = find_connection_payload(object_name, data)
                out_id = res[field_id]
                out_guid = res[field_guid]
                out_name = res[field_name]
                
                output.update({'Parent'+object_name+'ID':out_id,'Parent'+object_name+'Name':out_name, 'Parent'+object_name+'KnackGUID':out_guid})
    
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def getMoveTo(record, object_name):
    if object_name == 'LocalDuesCategory':
        phelper = 'Affiliate'
    else:
        phelper = 'Employer'
        
    output = {'UpdateIndividual'+phelper+'ToID':'','UpdateIndividual'+phelper+'ToName':'', 'UpdateIndividual'+phelper+'ToKnackGUID':''}
    conn_values = knackmappingdict[object_name]['move_to_fields']
    for k,v in conn_values.items():
        if 'MoveTo' in k:
            fieldval = v.replace('_raw', '')
            if record[fieldval]:
                data = record[v][0]['id']
                
                field_id = knackmappingdict[object_name]['ID']
                field_guid = knackmappingdict[object_name]['GUID']
                field_name = knackmappingdict[object_name]['fields'][(object_name+'Name')]
                
                res = find_connection_payload(object_name, data)
                out_id = res[field_id]
                out_guid = res[field_guid]
                out_name = res[field_name]
                
                output.update({'UpdateIndividual'+phelper+'ToID':out_id,'UpdateIndividual'+phelper+'ToName':out_name, 'UpdateIndividual'+phelper+'ToKnackGUID':out_guid})
    
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def payload_runner(entity_name, mapped, moveTo, parent):
    payload = find_records_updated_at_date(entity_name, DATE_TO_RUN)
    out = []
    #print(yaml.dump(payload, default_flow_style=False))
    for record in payload:
        fields = getFields(record, entity_name)
        connections = getConnections(record, entity_name)
        output1 = {**fields, **connections}

        if (mapped):
            
            # MAPPED FIELDS
            output2 = getMappedFields(record, entity_name)
            output1 = {**output1, **output2}
        if (moveTo):
            
            # MOVE TO FIELDS
            output2 = getMoveTo(record, entity_name)
            output1 = {**output1, **output2}
        if (parent):
            
            # PARENT FIELDS
            output2 = getParent(record, entity_name)
            output1 = {**output1, **output2}
        
        if fields:
            out.append(output1)
    return pd.DataFrame(out, dtype=str) 
            
def entity_runner(entity):
    print('Running Entity: '+entity+'...')

    extras = sorting_dict[entity]['extras_array']
    sort = sorting_dict[entity]['sort_array']

    output_df = payload_runner(entity, extras[0], extras[1], extras[2])
    rows = output_df.shape[0]
    columns = output_df.shape[1]

    if rows == 0 or columns == 0:
        print('No Changes found for '+entity+'!')
        print('Skipping write to civis...')
    else:
        print('Found '+str(rows)+' rows!')

        output_df = output_df[sort]
        entity = entity.lower()
        table_name = os.environ['table_schema'] + entity

        print('Fake writing to civis table: ' + table_name + '...')
        print(output_df)
        print('Done with debug output!')

def main():
    list_to_run =['LocalDuesCategory','Employer','LocalAgreement','Unit','LocalJobClass','JobTitle','WorkLocation','WorkStructure']
    for entity in list_to_run:
        print("================================")
        entity_runner(entity)
    print("================================")

main()

In [111]:
pip install yaml

ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.1.14_1/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
